In [59]:
import csv, json, unicodedata, requests, bs4

In [63]:
prefix = "http://volby.cz/pls/senat/se2111?xjazyk=CZ&xdatum="
#radne = ['19961116', '19981114', '20001112', '20021025', '20041105', '20061020', 
#         '20081017', '20121012', '20101015', '20141010', '20161007', '20181005']
radne = ["20181005"]
obvody = [i+1 for i in range(81)]
obvody1 = [i-1 for i in range(81) if not((i+1)%3)]
obvody2 = [i+1 for i in obvody1]
obvody3 = [i+1 for i in obvody2]
print(obvody, obvody1, obvody2, obvody3)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81] [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79] [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80] [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81]


In [64]:
urls = []
sep = "&xobvod="
for term in radne:
    rok = term[0:4]
    if rok == "1996": #prvni volby
        for obvod in obvody:
            urls.append(term+sep+str(obvod))
    elif not(int(rok) % 6): # prvni tretina
        for obvod in obvody1:
            urls.append(term+sep+str(obvod))
    elif not((int(rok)+4) % 6): # druha tretina
        for obvod in obvody2:
            urls.append(term+sep+str(obvod))
    elif not((int(rok)+2) % 6): # treti tretina
        for obvod in obvody3:
            urls.append(term+sep+str(obvod))
urls = sorted(urls, key=lambda x: int(x[0:4]))

#doplnovaci
urls.extend(["19990828&xobvod=27", "20031031&xobvod=12", "20031107&xobvod=58", "20041008&xobvod=20", "20041008&xobvod=54",
             "20070427&xobvod=63", "20070413&xobvod=5", "20110318&xobvod=30", "20140919&xobvod=22", "20140110&xobvod=80",
             "20180105&xobvod=39", "20180518&xobvod=78"])

In [66]:
senatori = []
for url in urls:
    datum, obvod = url.split("&xobvod=")
    content = requests.get(prefix+url).text.split("\n")
    for index, line in enumerate(content):
        if "*" in line and "d3.interpolate" not in line:
            jmeno, strana = bs4.BeautifulSoup(content[index+1]).text, bs4.BeautifulSoup(content[index+3]).text
            break
    print([datum,obvod,jmeno,strana])
    senatori.append([datum,obvod,jmeno,strana])

['20181005', '2', 'Balatka Miroslav Ing.', 'STAN']
['20181005', '5', 'Balatka Miroslav Ing.', 'STAN']
['20181005', '8', 'Balatka Miroslav Ing.', 'STAN']
['20181005', '11', 'Vilímec Vladislav Ing.', 'ODS']
['20181005', '14', 'Vilímec Vladislav Ing.', 'ODS']
['20181005', '17', 'Vilímec Vladislav Ing.', 'ODS']
['20181005', '20', 'Drahoš Jiří prof. Ing. DrSc.', 'STAN']
['20181005', '23', 'Drahoš Jiří prof. Ing. DrSc.', 'STAN']
['20181005', '26', 'Drahoš Jiří prof. Ing. DrSc.', 'STAN']
['20181005', '29', 'Drahoš Jiří prof. Ing. DrSc.', 'STAN']
['20181005', '32', 'Kubera Jaroslav', 'ODS']
['20181005', '35', 'Kubera Jaroslav', 'ODS']
['20181005', '38', 'Kubera Jaroslav', 'ODS']
['20181005', '41', 'Kubera Jaroslav', 'ODS']
['20181005', '44', 'Kubera Jaroslav', 'ODS']
['20181005', '47', 'Kubera Jaroslav', 'ODS']
['20181005', '50', 'Kubera Jaroslav', 'ODS']
['20181005', '53', 'Kubera Jaroslav', 'ODS']
['20181005', '56', 'Kubera Jaroslav', 'ODS']
['20181005', '59', 'Kubera Jaroslav', 'ODS']
['201

In [67]:
with open("senatori2018.csv","w",encoding="utf-8",newline="") as file:
    writer = csv.writer(file)
    headers = ["datum","obvod","vitez","vitezStrana"]
    writer.writerow(headers)
    writer.writerows(senatori)

In [70]:
output = {}
for filename in ("senatori.csv", "mapa.topo.json", "praha.topo.json"):
    with open(filename,encoding="utf-8") as file:
        data = file.read()
        try:
            output[filename.split(".")[0]] = json.loads(data)
        except: 
            output[filename.split(".")[0]] = data

with open("data.js","w",encoding="utf-8") as file:
    file.write("window.ig.data = " + json.dumps(output, ensure_ascii=False))